# Gai/Gen: Text-to-Code (TTC)

## 1.1 Setting Up

We will create a seperate virtual environment for this to avoid conflicting dependencies that each underlying model requires.

```sh
conda create -n TTC python=3.10.10 -y
conda activate TTC
pip install -e ".[TTC]"
```

NOTE: The installation depends on requirements_ttc.txt which is based on https://raw.githubusercontent.com/deepseek-ai/DeepSeek-Coder/main/requirements.txt

Download the deepseek coder model below.

In [ ]:
%%bash
huggingface-cli download deepseek-ai/deepseek-coder-6.7b-instruct \
        --local-dir ~/gai/models/deepseek-coder-6.7b-instruct \
        --local-dir-use-symlinks False

## 1.2 Running as a Library

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os
model_path = os.path.expanduser("~/gai/models/deepseek-coder-6.7b-instruct")
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True, torch_dtype=torch.bfloat16).cuda()
input_text = "#write a quick sort algorithm"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_length=128)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/home/roylai/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:39<00:00, 19.79s/it]
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


In [ ]:
from gai.gen.ttc.TTC import TTC
ttc = TTC(generator_name="deepseek-transformers")
ttc.create("write a quick sort algorithm",max_new_tokens=100,stream=False)